In [ ]:
import os
import numpy as np
from monai.transforms import (
    LoadImaged,
    Rotated,
    SaveImaged,
    EnsureChannelFirstd,
)
from scipy.optimize import fsolve
import pandas as pd
from tqdm import tqdm

## 1) Duplikasikan directory data resize
## 2) Jalankan program acpcdetect dari NITRC (https://www.nitrc.org/projects/art) pada setiap directory kasus data 

#### Struktur directory hasil acpcdetect kurang lebih akan seperti ini  
acpcdetect-data/  
|-- 10001/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_struct_init.nii.gz  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_struct_init_ACPC.txt  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- (data-data lainnya)  
|-- 10002/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_struct_init.nii.gz  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_struct_init_ACPC.txt  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- (data-data lainnya)  
|-- 10003/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_struct_init.nii.gz  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_struct_init_ACPC.txt  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- (data-data lainnya)  
...

#### Contoh  hasil deteksi (10001_struct_init_ACPC.txt)  
Input volume: 10001_struct_init.nii

Estimated mid-sagittal plane: (-0.1425441x) + (0.9897766y) + (-0.0048599z) =  4.11859 (mm)  
-0.1425441 0.9897766 -0.0048599  4.11859

AC-PC distance = 21.587 mm

VSPS (i,j,k) voxel location:
134.7 133.9 164.1

AC (i,j,k) voxel location:
112.2 130.6 138.5

PC (i,j,k) voxel location:
139.0 134.5 123.7


## 3) Jalankan kode berikut

In [ ]:
# Root directory data
data_dir = "acpcdetect_data"

# Root directory output
output_dir = "reoriented_data"

In [ ]:
def calc_X_rot(theta, y_PC, z_PC, z_AC):
	return y_PC * np.sin(theta) + z_PC * np.cos(theta) - z_AC

def calc_Z_rot(theta, x_PC, x_AC, y2):
	return x_PC * np.cos(theta) - y2*np.sin(theta) - x_AC

def read_line(line):
	x = line[:5].replace(' ', '0').replace('-', '6666').replace('nan', '9999')
	y = line[6:11].replace(' ', '0').replace('-', '6666').replace('nan', '9999')
	z = line[12:17].replace(' ', '0').replace('-', '6666').replace('nan', '9999')
	return float(x), float(y), float(z)

In [ ]:
invalid_count = 0
for dir in os.listdir("Rotated-ACPCdetect"):
	output_dir = os.path.join("Rotated-ACPCDetect-Final", dir)
	txt_path = os.path.join("Rotated-ACPCdetect", dir, "struct_ACPC.txt")
	img_path = os.path.join("Resized", dir, f"{dir}_TOF-pre.nii.gz")
	label_path = os.path.join("Resized", dir, f"{dir}_aneurysms.nii.gz")

	with open(txt_path, "r") as file:
		lines = file.readlines()
		
	x_AC, y_AC, z_AC = read_line(lines[11])
	x_PC, y_PC, z_PC = read_line(lines[14])

	if (not ((x_AC <= 256) and (y_AC <= 256) and (z_AC <= 256) and (x_PC <= 256) and (y_PC <= 256) and (z_PC <= 256))):
		print(dir, "Invalid")
		invalid_count += 1
		continue

	os.makedirs(output_dir, exist_ok=True)
		
	theta = fsolve(calc_X_rot, 0, args=(y_PC, z_PC, z_AC))
	angle_X_deg = np.degrees(theta[0])
	angle_X_rad = np.radians(angle_X_deg)

	y2 = y_PC * np.cos(angle_X_rad) - z_PC * np.sin(angle_X_rad)
	z2 = y_PC * np.sin(angle_X_rad) + z_PC * np.cos(angle_X_rad)

	if (abs(z2-z_AC) > 0.0000001):
		angle_X_rad_alt = np.radians(360-angle_X_rad)
		y2_alt = y_PC * np.cos(angle_X_rad_alt) - z_PC * np.sin(angle_X_rad_alt)
		z2_alt = y_PC * np.sin(angle_X_rad_alt) + z_PC * np.cos(angle_X_rad_alt)
		if(abs(z2-z_AC) > abs(z2_alt-z_AC)):
			y2 = y2_alt
			z2 = z2_alt
			angle_X_rad = angle_X_rad_alt

	theta = fsolve(calc_Z_rot, 0, args=(x_PC, x_AC, y2))
	angle_Z_deg = np.degrees(theta[0])
	angle_Z_rad = np.radians(angle_Z_deg)

	x3 = x_PC * np.cos(angle_Z_rad) - y2 * np.sin(angle_Z_rad)
	y3 = x_PC * np.sin(angle_Z_rad) + y2 * np.cos(angle_Z_rad)

	if (abs(x3-x_AC) > 0.0000001):
		angle_Z_rad_alt = np.radians(360-angle_Z_deg)
		x3_alt = x_PC * np.cos(angle_Z_rad_alt) - y2 * np.sin(angle_Z_rad_alt)
		y3_alt = x_PC * np.sin(angle_Z_rad_alt) + y2 * np.cos(angle_Z_rad_alt)
		if(abs(x3-x_AC) > abs(x3_alt-x_AC)):
			x3 = x3_alt
			y3 = y3_alt
			angle_Z_rad = angle_Z_rad_alt
	
	loader = LoadImaged(keys=("image"), image_only=False)
	rotate = Rotated(keys=("image"), angle=(angle_X_rad, 0, angle_Z_rad))
	ensure_channel_first = EnsureChannelFirstd(keys=["image"])
	saver = SaveImaged(keys=["image"], output_dir=output_dir, output_postfix="", separate_folder=False)

	train_dict = {'image': img_path}
	label_dict = {'image': label_path}

	data_dict = loader(train_dict)
	data_dict = ensure_channel_first(data_dict)
	data_dict = rotate(data_dict)
	output_image_path = f"{dir}_TOF-pre.nii.gz"
	data_dict["image"].meta["filename_or_obj"] = output_image_path
	saver(data_dict)

	data_dict = loader(label_dict)
	data_dict = ensure_channel_first(data_dict)
	data_dict = rotate(data_dict)
	output_image_path = f"{dir}_aneurysms.nii.gz"
	data_dict["image"].meta["filename_or_obj"] = output_image_path
	saver(data_dict)

Struktur akhir data reorientasi :  

reoriented_data/  
|-- 10001/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_aneurysms.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10001_TOF-pre.nii  
|-- 10002/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_aneurysms.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10002_TOF-pre.nii  
|-- 10003/  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_aneurysms.nii  
|&nbsp;&nbsp;&nbsp;&nbsp;|-- 10003_TOF-pre.nii  
...